In [13]:
!pip install opencv-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 MB 17.5 MB/s eta 0:00:0000:0100:01


In [19]:
try:
    import cv2  # opencv-python - optional if using ffprobe
except ImportError:
    cv2 = None

import subprocess

def get_playback_duration(video_filepath, method='cv2'):  # pragma: no cover
    """
    Get video playback duration in seconds and fps
    "This epic classic car collection centres on co.webm"
    :param video_filepath: str, path to video file
    :param method: str, method cv2 or default ffprobe
    """
    if method == 'cv2':  # Use opencv-python
        video = cv2.VideoCapture(video_filepath)
 
        fps = video.get(cv2.CAP_PROP_FPS)
        frame_count = video.get(cv2.CAP_PROP_FRAME_COUNT)
        print(video)
        duration_seconds = frame_count / fps if fps else 0
    else:  # ffprobe
        result = subprocess.check_output(
            f'ffprobe -v quiet -show_streams -select_streams v:0 -of json "{video_filepath}"', shell=True).decode()
        fields = json.loads(result)['streams'][0]
        duration_seconds = fields['tags'].get('DURATION')
        fps = eval(fields.get('r_frame_rate'))
    return duration_seconds, fps

print(get_playback_duration('60MI23_33_A_HBB.mp4','ffprobe'))

/bin/sh: 1: ffprobe: not found


CalledProcessError: Command 'ffprobe -v quiet -show_streams -select_streams v:0 -of json "60MI23_33_A_HBB.mp4"' returned non-zero exit status 127.

In [2]:
# for showing images and videos
from IPython.display import HTML
from IPython.display import Image as ImageByte, display

def get_public_url_from_gcs(gcs_uri: str) -> str:
    return gcs_uri.replace("gs://", "https://storage.googleapis.com/").replace(
        " ", "%20"
    )

def display_video_from_gcs(
    gcs_uri: str = None,
) -> None:
    html_code = f"""
    <video width="640" height="480" controls>
        <source src="{get_public_url_from_gcs(gcs_uri)}" type="video/mp4">
        Your browser does not support the video tag.
    </video>
    """
    display(HTML(html_code))
    
video_path = "gs://github-repo/embeddings/getting_started_embeddings/UCF-101-subset/BrushingTeeth/v_BrushingTeeth_g01_c02.mp4"
video_path='highway_vehicles.mp4'
display_video_from_gcs(video_path)


In [113]:
from typing import Any, Dict, Iterable, List, Optional, Tuple, Union
from vertexai.vision_models import (
    Image as VMImage,
    MultiModalEmbeddingModel,
    MultiModalEmbeddingResponse,
    Video as VMVideo,
    VideoSegmentConfig,
)
mm_embedding_model = MultiModalEmbeddingModel.from_pretrained("multimodalembedding")

def get_video_embedding(
    video_path: str,
    dimension: Optional[int] = 1408,
    video_segment_config: Optional[VideoSegmentConfig] = None,
) -> List[float]:
    video = VMVideo.load_from_file(video_path)
    embedding = mm_embedding_model.get_embeddings(
        video=video,
       # dimension=dimension,
        video_segment_config=video_segment_config,
    )
    return [video_emb.embedding for video_emb in embedding.video_embeddings],embedding


video_emb,embedding = get_video_embedding(
    video_path=video_path,
)

In [115]:
embedding

MultiModalEmbeddingResponse(_prediction_response=Prediction(predictions=[{'videoEmbeddings': [{'startOffsetSec': 0.0, 'embedding': [-0.011429769, 0.0216635913, -0.00484448, -0.0174837187, -0.00420923112, 0.0231927577, -0.040387731, -0.007527004, 0.00466555404, 0.0110773109, -0.00781155191, 0.0125557203, -0.0257435199, 0.0717051253, -0.0167904589, 0.00495507568, -0.0194394886, -0.0280418638, -0.0157037526, 0.000842605194, -0.0672384351, -0.0120969787, 0.0687451363, 0.00381434872, -0.0210449565, 0.00845900737, 0.0144297704, 0.0358397886, 0.0130378967, 0.00557739893, 0.0182531774, 0.0200032815, 0.00407613534, -0.0145179434, -0.00354251848, -0.021704169, 0.0281941313, -0.0275290459, -0.00961747, 0.0023790861, -0.014050738, -0.0167932101, -0.017337732, -0.023070965, 0.00279945089, 0.0462840609, 0.0148999095, 0.0288476367, -0.02046136, -0.00486880261, 0.0183500294, -0.0096639907, -0.0121982135, 0.0087831188, 7.27247925e-06, 0.0237993877, -0.0106776115, -0.0281728096, -0.0137579562, 0.0012604

In [112]:
 [x.embedding for x in embedding.video_embeddings]

[[-0.0114297699,
  0.0216636304,
  -0.00484452536,
  -0.0174836554,
  -0.0042092572,
  0.0231928024,
  -0.0403877497,
  -0.00752703752,
  0.00466557,
  0.0110772839,
  -0.00781159382,
  0.0125557426,
  -0.0257434603,
  0.0717049539,
  -0.016790444,
  0.00495510176,
  -0.0194394626,
  -0.0280418545,
  -0.0157038346,
  0.00084262609,
  -0.067238465,
  -0.0120969284,
  0.0687451363,
  0.00381431682,
  -0.0210449882,
  0.00845903344,
  0.0144298226,
  0.03583977,
  0.0130378772,
  0.00557739194,
  0.0182531923,
  0.0200033169,
  0.00407614745,
  -0.0145179424,
  -0.00354251848,
  -0.0217042174,
  0.0281941444,
  -0.0275290255,
  -0.00961747579,
  0.00237907725,
  -0.0140507584,
  -0.0167932454,
  -0.0173377153,
  -0.0230709892,
  0.00279943668,
  0.0462840609,
  0.0148999169,
  0.0288476311,
  -0.0204613581,
  -0.00486884592,
  0.0183500536,
  -0.00966402,
  -0.01219822,
  0.00878313836,
  7.30610236e-06,
  0.0237993654,
  -0.0106776683,
  -0.0281727966,
  -0.0137579422,
  0.00126040762,
 

In [109]:
embedding.videoEmbeddings

MultiModalEmbeddingResponse(_prediction_response=Prediction(predictions=[{'videoEmbeddings': [{'startOffsetSec': 0.0, 'embedding': [-0.0114297699, 0.0216636304, -0.00484452536, -0.0174836554, -0.0042092572, 0.0231928024, -0.0403877497, -0.00752703752, 0.00466557, 0.0110772839, -0.00781159382, 0.0125557426, -0.0257434603, 0.0717049539, -0.016790444, 0.00495510176, -0.0194394626, -0.0280418545, -0.0157038346, 0.00084262609, -0.067238465, -0.0120969284, 0.0687451363, 0.00381431682, -0.0210449882, 0.00845903344, 0.0144298226, 0.03583977, 0.0130378772, 0.00557739194, 0.0182531923, 0.0200033169, 0.00407614745, -0.0145179424, -0.00354251848, -0.0217042174, 0.0281941444, -0.0275290255, -0.00961747579, 0.00237907725, -0.0140507584, -0.0167932454, -0.0173377153, -0.0230709892, 0.00279943668, 0.0462840609, 0.0148999169, 0.0288476311, -0.0204613581, -0.00486884592, 0.0183500536, -0.00966402, -0.01219822, 0.00878313836, 7.30610236e-06, 0.0237993654, -0.0106776683, -0.0281727966, -0.0137579422, 0.00

In [102]:
[v.embedding for v in embedding.video_embeddings]

[[-0.0114297699,
  0.0216636304,
  -0.00484452536,
  -0.0174836554,
  -0.0042092572,
  0.0231928024,
  -0.0403877497,
  -0.00752703752,
  0.00466557,
  0.0110772839,
  -0.00781159382,
  0.0125557426,
  -0.0257434603,
  0.0717049539,
  -0.016790444,
  0.00495510176,
  -0.0194394626,
  -0.0280418545,
  -0.0157038346,
  0.00084262609,
  -0.067238465,
  -0.0120969284,
  0.0687451363,
  0.00381431682,
  -0.0210449882,
  0.00845903344,
  0.0144298226,
  0.03583977,
  0.0130378772,
  0.00557739194,
  0.0182531923,
  0.0200033169,
  0.00407614745,
  -0.0145179424,
  -0.00354251848,
  -0.0217042174,
  0.0281941444,
  -0.0275290255,
  -0.00961747579,
  0.00237907725,
  -0.0140507584,
  -0.0167932454,
  -0.0173377153,
  -0.0230709892,
  0.00279943668,
  0.0462840609,
  0.0148999169,
  0.0288476311,
  -0.0204613581,
  -0.00486884592,
  0.0183500536,
  -0.00966402,
  -0.01219822,
  0.00878313836,
  7.30610236e-06,
  0.0237993654,
  -0.0106776683,
  -0.0281727966,
  -0.0137579422,
  0.00126040762,
 

### generate video summary

In [62]:
from vertexai.preview.generative_models import (
    GenerationConfig,
    GenerativeModel,
    HarmCategory,
    HarmBlockThreshold,
    Part,
)

def get_url_from_gcs(gcs_uri: str) -> str:
    # converts GCS uri to url for display.
    return gcs_uri.replace("gs://", "https://storage.googleapis.com/").replace(
        " ", "%20"
    )

# Load file directly from Google Cloud Storage
video_uri = "gs://github-repo/video/KennedyNixon1960PresidentialDebate.mp4"

video_analysis_prompt=  """You are an assistant tasked with summarizing videos for retrieval. \
        These summaries will be embedded and used to retrieve the raw video. \
        Give a concise summary of the video that is well optimized for retrieval."""
promt='Chapterize the video content by grouping the video content into chapters and providing a summary for each chapter. Please only capture key events and highlights. If you are not sure about any info, please do not make it up. Return the result in the JSON format with keys as follows : "timecode", "chapterSummary"'
# Load contents
contents = [
    Part.from_uri(
        uri=video_uri,
        mime_type="video/mp4",
    ),
    video_analysis_prompt,
]

# Display the Video
display(Video(get_url_from_gcs(video_uri)))

generate=0
if generate==1:
 

    #model = GenerativeModel("gemini-1.5-pro-001")

    generation_config = GenerationConfig(temperature=1, top_p=0.95, max_output_tokens=8192)

    safety_settings = {
        HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_ONLY_HIGH,
        HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
        HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
        HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
    }

  

    # Send to Gemini
    response = model.generate_content(contents, generation_config=generation_config)

    # Display results
    display(Markdown(response.text))



In [74]:
import subprocess
import cv2 as cv
v = cv.VideoCapture('https://storage.cloud.google.com/raw_nine_files/60MI23_33_A_HBB.mp4')
v.set(cv.CAP_PROP_POS_AVI_RATIO, 1)

v.get(cv.CAP_PROP_POS_MSEC)

[mov,mp4,m4a,3gp,3g2,mj2 @ 0x556b87078000] moov atom not found


0.0

In [116]:
import tempfile, shutil

# Create temporary file to write embeddings to
videoembeddings_file = tempfile.NamedTemporaryFile(suffix=".json", delete=False)

 

In [124]:
embeddings_formatted

['{"id": "highway_vehicles.mp4", "embedding": ["-0.0229228195", "0.0329284146", "-0.0298541896", "0.00872737728", "-0.00977272075", "-0.0139661422", "-0.0184248388", "-0.0109950509", "-0.00882825442", "-0.0184693", "-0.00773923704", "0.000920513587", "0.0193172283", "0.0346956067", "-0.0380158275", "0.0246732589", "0.0014176179", "0.0152657991", "-0.0230893884", "-0.0160888322", "-0.0806713104", "0.017319845", "-0.0205730237", "0.00573240407", "-0.0463672802", "0.0288616829", "0.00946808606", "0.00292697828", "-0.00839182176", "0.00376370503", "0.0119599821", "0.0102707995", "0.0120545104", "0.000914060918", "0.00188469887", "-0.0199285299", "-0.0379240103", "-0.0292622298", "-0.0117192073", "0.042521406", "2.48530341e-05", "0.00704534911", "0.0163272507", "0.00909066852", "-0.0177795906", "-0.0343304165", "0.00705730123", "-0.00169996487", "0.0343478695", "0.0264834445", "-0.0381747074", "-0.0320935696", "0.017093366", "-0.00829951745", "-0.000441183918", "-0.00205723546", "-0.0304579

In [141]:
import vertexai
import json

from vertexai.vision_models import Image, MultiModalEmbeddingModel, Video
from vertexai.vision_models import VideoSegmentConfig


model = MultiModalEmbeddingModel.from_pretrained("multimodalembedding")

 

video = Video.load_from_file(
    "gs://cloud-samples-data/vertex-ai-vision/highway_vehicles.mp4"
)
contextual_text = "Cars on Highway"

max_video_duration=18 #second

prev=0
offsets=[]
for val in range (5,max_video_duration+5,5):
    offsets.append({'start':prev, 'end':val})
    prev=val
    
id='highway_vehicles.mp4'
results=[]
c=0
embeddings_formatted=[]
for offset in offsets:
    try:
        print(offset['start'],offset['end'])
        #---------------commented
        # embeddings = model.get_embeddings(
        #     #image=image,
        #     video=video,
        #     video_segment_config=VideoSegmentConfig(start_offset_sec=offset['start'],end_offset_sec=offset['end']),
        #     contextual_text=contextual_text,
        # )
        
        #-----------------
        embeddings=var_all_embeddings[c]
        #print(f"Image Embedding: {embeddings.image_embedding}")

        # Video Embeddings are segmented based on the video_segment_config.
        print("Video Embeddings:")
        #for video_embedding in embeddings.video_embeddings:
            # print(
            #     f"Video Segment: {video_embedding.start_offset_sec} - {video_embedding.end_offset_sec}"
            # )
            # print(f"Embedding: {video_embedding.embedding}")
            
        for  video_embedding in  embeddings.video_embeddings :
            if video_embedding is not None:
                embeddings_formatted.append( 
                    json.dumps(
                        {
                            "id": str(id),
                            "embedding": [str(value) for value in video_embedding.embedding],
                            "Video Segment ": f"{video_embedding.start_offset_sec} - {video_embedding.end_offset_sec}",
                        }
                    )
                    + "\n"
                   
                )
      
     
        
        results.append(embeddings)
        
        
        c=c+1
    except Exception as e:
        if 'InvalidArgument: 400 Multimodal embedding failed with the following error' in str(e):
            print('***Video Embedding ended at '+str(offset['start']))
            break
        else:
            print(e)
            break
    
    
with open(videoembeddings_file.name, "a") as ef:
    ef.writelines(embeddings_formatted)

0 5
Video Embeddings:
5 10
Video Embeddings:
10 15
Video Embeddings:
15 20
list index out of range


In [142]:
#save the temp files in persistent disk
import tempfile, shutil
 
file_name = videoembeddings_file.name
videoembeddings_file.close()
shutil.copy(file_name, 'videoembeddings_file.json')
 

'videoembeddings_file.json'

In [39]:
MP4Box SYD-NINE_NNNTW24_21_A.mp4 test.mp4

/bin/bash: line 1: MP4Box: command not found


In [92]:
!ffprobe -v quiet -show_streams -select_streams v:0 -of json "highway_vehicles.mp4"

{
    "streams": [
        {
            "index": 0,
            "codec_name": "h264",
            "codec_long_name": "H.264 / AVC / MPEG-4 AVC / MPEG-4 part 10",
            "profile": "High",
            "codec_type": "video",
            "codec_time_base": "1001/60000",
            "codec_tag_string": "avc1",
            "codec_tag": "0x31637661",
            "width": 1280,
            "height": 720,
            "coded_width": 1280,
            "coded_height": 720,
            "closed_captions": 0,
            "has_b_frames": 2,
            "pix_fmt": "yuv420p",
            "level": 32,
            "color_range": "tv",
            "color_space": "bt709",
            "color_transfer": "bt709",
            "color_primaries": "bt709",
            "chroma_location": "left",
            "refs": 1,
            "is_avc": "true",
            "nal_length_size": "4",
            "r_frame_rate": "30000/1001",
            "avg_frame_rate": "30000/1001",
            "time_base": "1/30000",
     

In [91]:
import os, sys, subprocess, shlex, re
from subprocess import call
def probe_file(filename):
    cmnd = ['ffprobe', '-show_format', '-pretty', '-loglevel', 'quiet', filename]
    p = subprocess.Popen(cmnd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    print (filename)
    out, err =  p.communicate()
    print ("==========output==========")
    print (out)
    if err:
        print( "========= error ========")
        print (err)

probe_file('60MI23_33_A_HBB.mp4')

60MI23_33_A_HBB.mp4
==========output==========
b''


In [98]:
import vertexai

from vertexai.vision_models import Image, MultiModalEmbeddingModel, Video
from vertexai.vision_models import VideoSegmentConfig

 

 
max_video_duration=18 #second

prev=0
offsets=[]
for val in range (1,max_video_duration+1,1):
    offsets.append({'start':prev, 'end':val})
    prev=val
    
 
results=[]
for offset in offsets:
    try:
        print(offset['start'],offset['end'])
        embeddings =[]

        #print(f"Image Embedding: {embeddings.image_embedding}")

        # Video Embeddings are segmented based on the video_segment_config.
        print("Video Embeddings:")
        for video_embedding in embeddings.video_embeddings:
            print(
                f"Video Segment: {video_embedding.start_offset_sec} - {video_embedding.end_offset_sec}"
            )
            print(f"Embedding: {video_embedding.embedding}")

        results.append(embeddings)
    except Exception as e:
        if 'InvalidArgument: 400 Multimodal embedding failed with the following error' in str(e):
            print('***')
        elif "'list' object has no attribute 'video_embeddings'" in str(e):
            print(e)
            break
        else:
            print(e)
            break
    
    


0 1
Video Embeddings:
'list' object has no attribute 'video_embeddings'


In [144]:
def load_image_bytes(image_uri: str) -> bytes:
    """Load image bytes from a remote or local URI."""
    image_bytes = None
    if image_uri.startswith("http://") or image_uri.startswith("https://"):
        response = requests.get(image_uri, stream=True)
        if response.status_code == 200:
            image_bytes = response.content
    else:
        image_bytes = open(image_uri, "rb").read()
    return image_bytes

In [5]:
from vertexai.preview.generative_models import (
    GenerationConfig,
    GenerativeModel,
    HarmCategory,
    HarmBlockThreshold,
    Part,
)

generative_multimodal_model= GenerativeModel("gemini-1.5-flash-001")
        
dir(generative_multimodal_model.generate_content)

['__call__',
 '__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__func__',
 '__ge__',
 '__get__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__self__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

In [ ]:
load_image_bytes('highway_vehicles.mp4')

In [ ]:
   def get_video_summarycontent(self, text: str = None, video_file: str = None):
        
         '''
         Gimini Features for Video as per:https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/video-understanding
         gemini-1.5-flash, Gemini 1.5 Pro: 
         With audio: ~50 minutes
         Without audio: 60 minutes
         Maximum videos per prompt: 10
         
         Gemini 1.0 Pro Vision:
         Maximum video length: 2 minutes
         The maximum videos per prompt: 1
         Audio in the video is ignored.
        '''
        

        generative_multimodal_model= GenerativeModel("gemini-1.5-flash-001")
        
        # Please only capture key events and highlights.
        video_description_prompt="""You are an assistant tasked with summarizing videos for retrieval.\
         These summaries will be embedded and used to retrieve the raw video.\
        Chapterize the video content by grouping the video content into chapters \
        and providing a concise summary for each chapter that is well optimized for retrieval.\
        If there is a famous person like politician, celebrity or athlete, indicate their name and describe what they are famous for.\
        If you are not sure about any info, please do not make it up. \
        Return the result in the JSON format with keys as follows : "timecode", "chapterSummary"
        """

        generation_config= GenerationConfig(temperature=1, max_output_tokens=8192,TopK=40,TopP=0.95) 
        
        safety_settings=  {
        HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
        }
        stream=True
        
        # Load the saved video as a Gemini Video Object
        #video_for_gemini= Video.load_from_file(video_file)
        video=Part
        model_input=[video_description_prompt, video_for_gemini]
        
        response = generative_multimodal_model.generate_content(
        model_input,
        generation_config=generation_config,
        stream=stream,
        safety_settings=safety_settings, )
        
        
        response_list = []

        for chunk in response:
            try:
                response_list.append(chunk.text)
            except Exception as e:
                print(
                    "Exception occurred while calling gemini. Something is wrong. Lower the safety thresholds [safety_settings: BLOCK_NONE ] if not already done. -----",
                    e,
                )
                response_list.append("Exception occurred")
                continue
        response = "".join(response_list)
 
        return response

In [ ]:
from google.cloud import storage
client = storage.Client()
BUCKET_NAME = 'DEMO_BUCKET'
bucket = client.get_bucket(BUCKET_NAME)

blobs = bucket.list_blobs()

for blob in blobs:
    print(blob.name)

In [1]:
from google.cloud import storage

client = storage.Client()
for blob in client.list_blobs('bucketname', prefix='raw_nine_files'):
  print(str(blob))

Forbidden: 403 GET https://storage.googleapis.com/storage/v1/b/bucketname/o?projection=noAcl&prefix=raw_nine_files&prettyPrint=false: 494586852359-compute@developer.gserviceaccount.com does not have storage.objects.list access to the Google Cloud Storage bucket. Permission 'storage.objects.list' denied on resource (or it may not exist).

In [1]:
import ffmpeg
vid = ffmpeg.probe('60MI23_33_A_HBB.mp4')
print(vid['streams'])

Error: ffprobe error (see stderr output for detail)